In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# !pip install git+https://github.com/rcmalli/keras-vggface.git

# Any results you write to the current directory are saved as output.

In [ ]:
from collections import defaultdict
from glob import glob
from random import choice, sample
from keras.preprocessing import image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from keras.models import Model
from keras.optimizers import Adam
# from keras_vggface.utils import preprocess_input
# from keras_vggface.vggface import VGGFace
from keras.utils.vis_utils import plot_model
import ssl
from keras.models import load_model

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
train_file_path = "../input/recognizing-faces-in-the-wild/train_relationships.csv"
train_folders_path = "../input/recognizing-faces-in-the-wild/train/"
val_famillies = "F09"

In [ ]:
all_images = glob(train_folders_path + "*/*/*.jpg")
train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]


In [ ]:
train_person_to_images_map = defaultdict(list)
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [ ]:
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]


In [ ]:
def read_img_facenet(path):
    img = image.load_img(path, target_size=(160, 160))
#     img = image.load_img(path)
    img = np.array(img).astype(np.float)
    return img

In [ ]:
def read_img_vggface(path):
#     img = image.load_img(path, target_size=(197, 197))
    img = image.load_img(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [ ]:
def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img_facenet(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img_facenet(x) for x in X2])

        yield [X1, X2], labels

In [ ]:
def baseline_model_facenet():
    input_1 = Input(shape=(160, 160, 3))
    input_2 = Input(shape=(160, 160, 3))
#     input_1 = Input((None, None, 3))
#     input_2 = Input((None, None, 3))
    base_model = load_model("../input/keras-facenet-modelweights/keras-facenet-20190718t123546z-001/keras-facenet/model/facenet_keras.h5")
#     base_model_trunc = Model([input_1, input_2], base_model.layers[-4].output)
    last_layer = base_model.get_layer('Bottleneck_BatchNorm').output
    base_model_trunc = Model(base_model.input, last_layer)
    for x in base_model_trunc.layers:
        x.trainable =True
    x1 = base_model_trunc(input_1)
    x2 = base_model_trunc(input_2)

#     x1_ = Reshape(target_shape=(7*7, 2048))(x1)
#     x2_ = Reshape(target_shape=(7*7, 2048))(x2)
#     #
#     x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
#     x_dot = Flatten()(x_dot)

#     x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
#     x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x = Concatenate(axis=-1)([x4, x3])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.01)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
def baseline_model_vggface():
#     input_1 = Input(shape=(197, 197, 3))
#     input_2 = Input(shape=(197, 197, 3))
    input_1 = Input((None, None, 3))
    input_2 = Input((None, None, 3))
    base_model = VGGFace(model='resnet50', include_top=False)
#     base_model_trunc = Model([input_1, input_2], base_model.layers[-4].output)
    last_layer = base_model.get_layer('conv5_3_1x1_increase/bn').output
    base_model_trunc = Model(base_model.input, last_layer)
    for x in base_model_trunc.layers:
        x.trainable =True
    x1 = base_model_trunc(input_1)
    x2 = base_model_trunc(input_2)

#     x1_ = Reshape(target_shape=(7*7, 2048))(x1)
#     x2_ = Reshape(target_shape=(7*7, 2048))(x2)
#     #
#     x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
#     x_dot = Flatten()(x_dot)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x = Concatenate(axis=-1)([x4, x3])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.01)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "facenet_finetuned.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model_facenet()


In [ ]:
for x in model.layers:
    print(x.name, x.trainable)

In [ ]:
history= model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=100, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)

In [ ]:
model.load_weights('facenet_finetuned.h5')

In [ ]:
test_path = "../input/recognizing-faces-in-the-wild/test/"


def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv('../input/recognizing-faces-in-the-wild/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img_facenet(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img_facenet(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("facenet_finetuned.csv", index=False)


In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
t = f.suptitle('Siamese Net Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)
epoch_list = history.epoch

ax1.plot(epoch_list, history.history['acc'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, epoch_list[-1], 5))
ax1.set_ylabel('Accuracy Value');ax1.set_xlabel('Epoch');ax1.set_title('Accuracy')
ax1.legend(loc="best");ax1.grid(color='gray', linestyle='-', linewidth=0.5)

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, epoch_list[-1], 5))
ax2.set_ylabel('Loss Value');ax2.set_xlabel('Epoch');ax2.set_title('Loss')
ax2.legend(loc="best");ax2.grid(color='gray', linestyle='-', linewidth=0.5)

# ax3.plot(epoch_list, history.history['auc'], label='Train AUC')
# ax3.plot(epoch_list, history.history['val_auc'], label='Validation AUC')
# ax3.set_xticks(np.arange(0, epoch_list[-1], 5))
# ax3.set_ylabel('AUC');ax3.set_xlabel('Epoch');ax3.set_title('AUC')
# ax3.legend(loc="best");ax3.grid(color='gray', linestyle='-', linewidth=0.5)


In [ ]:
! wget https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_tf_notop_resnet50.h5

<a href="facenet_finetuned.csv"> Download File </a>

In [ ]:
from keras.models import load_model
# load the model
model = load_model("../input/keras-facenet-modelweights/keras-facenet-20190718t123546z-001/keras-facenet/model/facenet_keras.h5")
# summarize input and output shape
print(model.inputs)
print(model.outputs)